In [1]:
import pandas as pd
import spacy
from spacy.tokenizer import Tokenizer
from spacy.symbols import ORTH

import data_preparation as mdp
from mercari_config import MercariConfig

In [2]:
%%time

nlp = spacy.load('en_core_web_md', disable=['parser', 'tagger', 'ner'])  
tokenizer = Tokenizer(nlp.vocab)

train_data = mdp.load_data(MercariConfig.TRAINING_SET_PREP_FILE)

CPU times: user 18.3 s, sys: 1.13 s, total: 19.5 s
Wall time: 19.4 s


In [3]:
nlp.pipeline

[]

In [4]:
nlp.tokenizer.add_special_case('[rm]', [{ORTH: '[rm]'}])

In [5]:
%%time

word = mdp.init_word_dict()
max_words_in_item_desc = mdp.walk_items(train_data['item_description'], nlp, word)

Progress: 0.42
Progress: 0.84
Progress: 1.26
Progress: 1.69
Progress: 2.11
Progress: 2.53
Progress: 2.95
Progress: 3.37
Progress: 3.79
Progress: 4.22
Progress: 4.64
Progress: 5.06
Progress: 5.48
Progress: 5.90
Progress: 6.32
Progress: 6.75
Progress: 7.17
Progress: 7.59
Progress: 8.01
Progress: 8.43
Progress: 8.85
Progress: 9.27
Progress: 9.70
Progress: 10.12
Progress: 10.54
Progress: 10.96
Progress: 11.38
Progress: 11.80
Progress: 12.23
Progress: 12.65
Progress: 13.07
Progress: 13.49
Progress: 13.91
Progress: 14.33
Progress: 14.76
Progress: 15.18
Progress: 15.60
Progress: 16.02
Progress: 16.44
Progress: 16.86
Progress: 17.28
Progress: 17.71
Progress: 18.13
Progress: 18.55
Progress: 18.97
Progress: 19.39
Progress: 19.81
Progress: 20.24
Progress: 20.66
Progress: 21.08
Progress: 21.50
Progress: 21.92
Progress: 22.34
Progress: 22.77
Progress: 23.19
Progress: 23.61
Progress: 24.03
Progress: 24.45
Progress: 24.87
Progress: 25.29
Progress: 25.72
Progress: 26.14
Progress: 26.56
Progress: 26.98

In [6]:
word2index = mdp.build_word2index(word=word)

In [7]:
word2index

,word_id,entity_type,count
word,,,
___PAD___,0,___NONE_ENTITY___,0
___START___,1,___NONE_ENTITY___,0
___OOV___,2,___NONE_ENTITY___,0
[rm],3,___NONE_ENTITY___,29567
___VERY_EMPTY_NAME___,4,___NONE_ENTITY___,0
___VERY_EMPTY_CATEGORY___,5,___NONE_ENTITY___,0
___VERY_EMPTY_BRAND___,6,___NONE_ENTITY___,0
___VERY_EMPTY_DESCRIPTION___,7,___NONE_ENTITY___,0
,8,___NONE_ENTITY___,2


In [8]:
mdp.save_word2index(word2index=word2index, file_name=MercariConfig.WORD_2_INDEX_4_ITEM_DESC_FILE )

In [9]:
len(word2index)

221368

In [10]:
word2index.count()

word_id        221368
entity_type    221368
count          221368
dtype: int64

In [11]:
max_words_in_item_desc

537

In [12]:
word2index.groupby("entity_type").count()

,word_id,count
entity_type,,
CARDINAL,10113,10113
DATE,7007,7007
EVENT,1176,1176
FAC,1515,1515
GPE,5019,5019
LANGUAGE,30,30
LAW,567,567
LOC,1018,1018
MONEY,3293,3293
